In [ ]:
#|default_exp cli

In [ ]:
#|export
from __future__ import annotations
import warnings

from nbdev.config import *
from nbdev.process import *
from nbdev.processors import *
from nbdev.doclinks import *
from nbdev.test import *
from nbdev.clean import *
from nbdev.quarto import refresh_quarto_yml
from nbdev.frontmatter import FrontmatterProc

from execnb.nbio import *
from fastcore.meta import *
from fastcore.utils import *
from fastcore.script import call_parse
from fastcore.style import S
from fastcore.shutil import rmtree,move

from urllib.error import HTTPError
from contextlib import redirect_stdout
import os, tarfile, sys

In [ ]:
#|hide
from nbdev import show_doc
from fastcore.test import *

# cli
> CLI commands

## Prepare -

In [ ]:
#|export
@call_parse
def prepare():
    "Export, test, and clean notebooks"
    nbdev_export.__wrapped__()
    nbdev_test.__wrapped__()
    nbdev_clean.__wrapped__()

## Filter -

In [ ]:
#|export
class FilterDefaults:
    "Override `FilterDefaults` to change which notebook processors are used"
    def xtra_procs(self): return []

    def base_procs(self):
        return [FrontmatterProc, populate_language, add_show_docs, insert_warning,
                strip_ansi, hide_line, filter_stream_, rm_header_dash,
                clean_show_doc, exec_show_docs, rm_export, clean_magics, hide_, add_links, strip_hidden_metadata]

    def procs(self):
        "Processors for export"
        return self.base_procs() + self.xtra_procs()
    
    def nb_proc(self, nb):
        "Get an `NBProcessor` with these processors"
        return NBProcessor(nb=nb, procs=self.procs())

In [ ]:
#|export
@call_parse
def nbdev_filter(
    nb_txt:str=None,  # Notebook text (uses stdin if not provided)
    fname:str=None,  # Notebook to read (uses `nb_txt` if not provided)
):
    "A notebook filter for Quarto"
    os.environ["IN_TEST"] = "1"
    try: filt = get_config().get('exporter', FilterDefaults)()
    except FileNotFoundError: filt = FilterDefaults()
    printit = False
    if fname: nb_txt = Path(fname).read_text()
    elif not nb_txt: nb_txt,printit = sys.stdin.read(),True
    nb = dict2nb(loads(nb_txt))
    if printit:
        with open(os.devnull, 'w') as dn:
            with redirect_stdout(dn): filt.nb_proc(nb).process()
    else: filt.nb_proc(nb).process()
    res = nb2str(nb)
    del os.environ["IN_TEST"]
    if printit: print(res, flush=True)
    else: return res

In [ ]:
#|hide
# print(nbdev_filter(fname='/Users/jhoward/git/nbdev/nbs/06_merge.ipynb'))

## New -

In [ ]:
#|export
def extract_tgz(url, dest='.'):
    from fastcore.net import urlopen
    with urlopen(url) as u: tarfile.open(mode='r:gz', fileobj=u).extractall(dest)

In [ ]:
#|export
def _render_nb(fn, cfg):
    "Render templated values like `{{lib_name}}` in notebook at `fn` from `cfg`"
    txt = fn.read_text()
    txt = txt.replace('from your_lib.core', f'from {cfg.lib_path}.core') # for compatibility with old templates
    for k,v in cfg.d.items(): txt = txt.replace('{{'+k+'}}', v)
    fn.write_text(txt)

In [ ]:
#|export
@call_parse
@delegates(nbdev_create_config)
def nbdev_new(**kwargs):
    "Create a new project."
    from fastcore.net import urljson
    
    nbdev_create_config.__wrapped__(**kwargs)
    cfg = get_config()

    path = Path()
    tag = urljson('https://api.github.com/repos/fastai/nbdev-template/releases/latest')['tag_name']
    url = f"https://github.com/fastai/nbdev-template/archive/{tag}.tar.gz"
    extract_tgz(url)
    tmpl_path = path/f'nbdev-template-{tag}'

    nbexists = bool(first(path.glob('*.ipynb')))
    for o in tmpl_path.ls():
        if o.name == 'index.ipynb': _render_nb(o, cfg)
        if o.name == '00_core.ipynb' and not nbexists: move(str(o), './')
        elif not (path/o.name).exists(): move(str(o), './')
    rmtree(tmpl_path)

    refresh_quarto_yml()

    nbdev_export.__wrapped__()

In [ ]:
#|hide
# NOTE: temporarily disabled tests until we make them more robust

In [ ]:
#|notest
#|hide
import nbdev; nbdev.nbdev_export() # Ensure we have the latest command below

In [ ]:
#|notest
#|hide
import tempfile

In [ ]:
#|notest
#|hide
# Ensure we're in an empty tempdir for testing
cwd = get_config().path('nbs_path')
try: rmtree(tmpdir)
except (NameError, FileNotFoundError): pass
tmpdir = Path(tempfile.mkdtemp())
p = tmpdir/'my-project'
p.mkdir()
os.chdir(p)

Since it calls `nbdev_create_config` internally, settings can be passed as command line args, and missing settings are inferred from the current git/GitHub repo if possible, otherwise prompted for.

In [ ]:
#|notest
!nbdev_new --repo my-project --user fastai --author fastai --author_email info@fast.ai --description 'A test project'

settings.ini created.


Your repo will now contain the following:

In [ ]:
#|notest
!ls -a

.             .gitignore    MANIFEST.in   index.ipynb   setup.py
..            00_core.ipynb README.md     my_project    styles.css
.github       LICENSE       _quarto.yml   settings.ini


Information will be rendered from your config into `index.ipynb`:

In [ ]:
#|notest
index_nb = read_nb('index.ipynb')
show_src(index_nb.cells[0].source)

```python
#| hide
from my_project.core import *
```

In [ ]:
#|notest
show_src(index_nb.cells[1].source)

```python
# my-project

> A test project
```

In [ ]:
#|notest
show_src(index_nb.cells[4].source.splitlines()[1], lang='sh')

```sh
pip install my_project
```

In [ ]:
#|notest
#|hide
# NOTE: temporarily disabled - it fails on linux but not mac
# `nbdev_prepare` succeeds with no file changes on a new nbdev repo
!rm -rf .git
!git init -q
!git add .
!git commit -q -m'Initial commit'
!nbdev_prepare
test_eq(run('git status -uno -s'), '')

Success.


In [ ]:
#|notest
#|hide
os.chdir(cwd) # Go back to original working dir
rmtree(tmpdir)

## Help

In [ ]:
#|export
@call_parse
def chelp():
    "Show help for all console scripts"
    from fastcore.xtras import console_help
    console_help('nbdev')

In [ ]:
chelp()

nbdev_bump_version              Increment version in settings.ini by one
nbdev_changelog                 Create a CHANGELOG.md file from closed and labeled GitHub issues
nbdev_clean                     Clean all notebooks in `fname` to avoid merge conflicts
nbdev_conda                     Create a `meta.yaml` file ready to be built into a package, and optionally build and upload it
nbdev_create_config             Create a config file.
nbdev_deploy                    Deploy docs to GitHub Pages
nbdev_docs                      Create Quarto docs and README.md
nbdev_export                    Export notebooks in `path` to Python modules
nbdev_filter                    A notebook filter for Quarto
nbdev_fix                       Create working notebook from conflicted notebook `nbname`
nbdev_help                      Show help for all console scripts
nbdev_install                   Install Quarto and the current library
nbdev_install_hooks             Install Jupyter and git hooks to automa

## Export -

In [ ]:
#|hide
import nbdev; nbdev.nbdev_export()